# Radar Chart

This notebook features a radar chart, inspired by an example [Radar chart with ggradar](https://r-graph-gallery.com/web-radar-chart-with-R.html). The data utilized is sourced from the [penguins dataset](https://github.com/JohnMount/Penguins/blob/main/penguins.csv), and is shared under the [CC0 license](https://allisonhorst.github.io/palmerpenguins/#license).

In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def rescale_in_group_df(df, value_col, group_col):
    def rescale_subdf(subdf):
        subdf.loc[:, value_col] -= subdf[value_col].min()
        subdf.loc[:, value_col] /= subdf[value_col].max()
        return subdf
    return pd.concat([rescale_subdf(df[df[group_col] == g]) for g in df[group_col].unique()])

def get_data():
    df = pd.read_csv("https://raw.githubusercontent.com/JohnMount/Penguins/main/penguins.csv")
    grouped_df = df.dropna().groupby("species")[["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]].mean().reset_index()
    grouped_df.columns = ["species", "average bill length", "average bill depth", "average flipper length", "average body mass"]
    melted_df = pd.melt(grouped_df, id_vars="species")
    rescaled_df = rescale_in_group_df(melted_df, "value", "variable")
    rescaled_df.loc[:, "value"] = (100 * rescaled_df["value"]).astype(int)
    return rescaled_df.sort_values(by="species").reset_index(drop=True)

In [4]:
penguins_df = get_data()

In [5]:
font_family = "roboto"
axis_color = "lightgray"
axis_text_data = {"x": ["average bill length"]*3, \
                  "y": [0, 50, 100], \
                  "text": ["0%", "50%", "100%"]}
colors_of_species = {"Adelie": "#ff5a5f", "Chinstrap": "#ffb400", "Gentoo": "#007a87"}
radar_tooltips = layer_tooltips().title("@species").line("@variable: @value%")
radar_theme = theme(
    text=element_text(family=font_family, size=18),
    plot_title=element_text(size=28, hjust=.5, face='bold'),
    axis_title='blank', axis_text_y='blank', axis_line_x='blank', \
    axis_ticks=element_line(color=axis_color), \
    panel_grid=element_line(color=axis_color), \
    panel_inset=[0, 80, 0, 80], \
    legend_position=[1, 0], legend_justification=[1, 0],
    axis_tooltip='blank',
)

In [6]:
ggplot(penguins_df) + \
    geom_area(aes("variable", "value", paint_a="species"), position='identity', flat=True, \
              size=2.5, color_by='paint_a', fill_by='paint_a', alpha=.2) + \
    geom_point(aes("variable", "value", paint_a="species"), size=6, color_by='paint_a', tooltips=radar_tooltips) + \
    geom_text(aes("x", "y", label="text"), data=axis_text_data, hjust=1, fontface='bold', family=font_family, size=10) + \
    scale_x_discrete() + \
    scale_manual('paint_a', name='', values=colors_of_species) + \
    coord_polar(ylim=[-15, 100]) + \
    ggsize(800, 600) + \
    ggtitle("Radar plot of penguins species") + \
    radar_theme + flavor_solarized_light()